In [1]:
import pandas as pd
from data_processing.encoding.categorical import CategoricalTransformer
from data_processing.encoding.numeric_and_date import ToQuantileTransformer
#from data_processing.encoding.text2vec import TextTransform

Load relevant data

In [2]:
data = pd.read_csv('data/other_data/PreFer_fake_data.csv')
codebook = pd.read_csv('data/codebooks/PreFer_codebook.csv')

Get the two relevant columns (Only core questions for now)

In [3]:
codebook = codebook[codebook.year.notna()]
categorical_columns = codebook[codebook.type_var == 'categorical'].var_name
quantile_columns = codebook[(codebook.type_var == 'numeric') | (codebook.type_var == 'date or time')].var_name

In [4]:
cat_transform = CategoricalTransformer()
cat_transform.fit(codebook)

This can be changed to work on a dataframe

In [5]:
data[categorical_columns] = cat_transform.transform(data[categorical_columns])

In [6]:
quantile_transform = ToQuantileTransformer(quantile_columns)
quantile_transform.fit(data)
data = quantile_transform.transform(data)

Replace nans and set all columns to int

In [7]:
data = data.fillna(101)
data = data.astype(int, errors='ignore')
data = data[data.columns[data.dtypes != 'object']]

In [8]:
from data_processing.sequences.sequencing import to_sequences

sequences = to_sequences(data, codebook)

Check if everything is correct

In [9]:
max_vocab = max(cat_transform.vocab.values())

In [10]:
for pid, patient in sequences.items():
    for year, seq in patient.items():
        for x in seq:
            if not isinstance(x, int):
                print(x)
            if x > max_vocab or x < 0:
                print(x)